This project utilizes the PokeAPI (https://pokeapi.co/) to create a large database containing relvent information on Pokemon to be used for future projects. Note: This is not complete - Work In Progress

### Import the required Libraries

In [1]:
import requests
import pandas as pd
import time

### Create The Inital pd dataframes

In [2]:
pokedf =  pd.DataFrame(columns = ['name','species', 'height', 'weight', 'abilities', 'types', 'stats'])
specdf = pd.DataFrame(columns = ['name','pokedex','gen','genus', 'capture_rate', 'color', 'shape', 'egg_groups', 'is_baby', 'is_legendary', 'is_mythical'])

### Functions to request Data

In [3]:
def get_pokemon_info(url):
    response = requests.get(url)
    
    if response.status_code == 200:
        pokemon_data = response.json()
        pokemon_info = {
            "name": pokemon_data["name"],
            "species": pokemon_data["species"]["name"],
            "height": pokemon_data["height"],
            "weight": pokemon_data["weight"],
            "abilities": [ability["ability"]["name"] for ability in pokemon_data["abilities"]],
            "types": [stats_data["type"]["name"] for stats_data in pokemon_data["types"]],
            "stats": [stats_data["base_stat"] for stats_data in pokemon_data["stats"]],
            #"hp": pokemon_data["stats"][0]["base_stat"],
            #"attack": pokemon_data["stats"][1]["base_stat"],
            #"defence": pokemon_data["stats"][2]["base_stat"],
            #"sp_attack": pokemon_data["stats"][3]["base_stat"],
            #"sp_defence": pokemon_data["stats"][4]["base_stat"],
            #"speed": pokemon_data["stats"][5]["base_stat"],
        }
        pokedf.loc[len(pokedf)] = pokemon_info
        return None
    else:
        return None

In [4]:
def get_species_info(url):
    response = requests.get(url)
    
    if response.status_code == 200:
        species_data = response.json()
        species_info = {
            "name": species_data["name"],
            "pokedex": species_data["pokedex_numbers"][0]["entry_number"],
            "gen": species_data["generation"]["name"],
            #"genus": species_data["genera"]["genus"],
            "capture_rate": species_data["capture_rate"],
            "color": species_data["color"]["name"],
            #"shape": species_data["shape"]["name"],
            "egg_groups": [group["name"] for group in species_data["egg_groups"]],
            "is_baby": species_data["is_baby"],
            "is_legendary": species_data["is_legendary"],
            "is_mythical": species_data["is_mythical"],            
        }
        specdf.loc[len(specdf)] = species_info
        return None
    else:
        return None

### WebScraping to Create the DataFrames

In [5]:
t0 = time.time()
# Pokemon df
url = f"https://pokeapi.co/api/v2/pokemon?limit=100000&offset=0"    #Real URL
#url = f"https://pokeapi.co/api/v2/pokemon?limit=50&offset=0"        #test URL
response = requests.get(url)
poke = response.json()
for pokemon in poke['results']:
    get_pokemon_info(pokemon['url'])
    
t1 = time.time()
timer = t1-t0
print(f"time elapsed: {timer/60} minutes")

time elapsed: 12.163010708491008 minutes


In [6]:
t0 = time.time()
#Species df
url = f"https://pokeapi.co/api/v2/pokemon-species?limit=100000&offset=0"   #Real URL
#url = f"https://pokeapi.co/api/v2/pokemon-species?limit=50&offset=0"    #test URL
response = requests.get(url)
species = response.json()
for spec in species['results']:
    get_species_info(spec['url'])

t1 = time.time()
timer = t1-t0
print(f"time elapsed: {timer/60} minutes")

time elapsed: 9.792537236213684 minutes


In [7]:
#Backup df
pokedf_backup = pokedf.copy()
specdf_backup = specdf.copy()

### Function to edit generation format (Data Cleaning)

In [8]:
def get_generation(generation):
    if generation == 'generation-i':
        return 1
    elif generation == 'generation-ii':
        return 2
    elif generation == 'generation-iii':
        return 3
    elif generation == 'generation-iv':
        return 4
    elif generation == 'generation-v':
        return 5
    elif generation == 'generation-vi':
        return 6
    elif generation == 'generation-vii':
        return 7
    elif generation == 'generation-viii':
        return 8
    elif generation == 'generation-ix':
        return 9
    else:
        return "NA"

# Manipulating Dataframes  


In [9]:
#Spread out Types/Abilities/Stats
pokedf[['hp', 'attack', 'defence', 'sp_attack', 'sp_defence', 'speed']] = pd.DataFrame(pokedf.stats.tolist(), index= pokedf.index)
pokedf[['type1','type2']] = pd.DataFrame(pokedf.types.tolist(), index= pokedf.index)
pokedf[['ability1','ability2', 'hidden_ability']] = pd.DataFrame(pokedf.abilities.tolist(), index= pokedf.index)

#Pop Original Columns
stats_col = pokedf.pop('stats')
types_col = pokedf.pop('types')
abilities_col = pokedf.pop('abilities')

'\nDrop Columns of lists\npokedf = pokedf.drop("types", axis = 1)\npokedf = pokedf.drop("abilities", axis = 1)\npokedf = pokedf.drop("stats", axis = 1)\n'

In [10]:
#Format Generation
specdf["generation"] = specdf["gen"].apply(get_generation)
del specdf['gen']

#Spread out egg groups
specdf[['egg_group1','egg_group2']] = pd.DataFrame(specdf.egg_groups.tolist(), index= specdf.index)
egg_groups_col = specdf.pop('egg_groups')

In [11]:
#Merge the dataframes
df2 = pd.merge(pokedf, specdf, how="left", left_on='species', right_on='name')
del df2['name_y']
df2 = df2.rename(columns={'name_x': 'name'})

In [12]:
df2['is_mega'] = pd.DataFrame(df2.name.str.contains("-mega"))
df2.head(20)

,name,species,height,weight,hp,attack,defence,sp_attack,sp_defence,speed,...,capture_rate,color,shape,is_baby,is_legendary,is_mythical,generation,egg_group1,egg_group2,is_mega
0,bulbasaur,bulbasaur,7,69,45,49,49,65,65,45,...,45,green,NaN,False,False,False,1,monster,plant,False
1,ivysaur,ivysaur,10,130,60,62,63,80,80,60,...,45,green,NaN,False,False,False,1,monster,plant,False
2,venusaur,venusaur,20,1000,80,82,83,100,100,80,...,45,green,NaN,False,False,False,1,monster,plant,False
3,charmander,charmander,6,85,39,52,43,60,50,65,...,45,red,NaN,False,False,False,1,monster,dragon,False
4,charmeleon,charmeleon,11,190,58,64,58,80,65,80,...,45,red,NaN,False,False,False,1,monster,dragon,False
5,charizard,charizard,17,905,78,84,78,109,85,100,...,45,red,NaN,False,False,False,1,monster,dragon,False
6,squirtle,squirtle,5,90,44,48,65,50,64,43,...,45,blue,NaN,False,False,False,1,monster,water1,False
7,wartortle,wartortle,10,225,59,63,80,65,80,58,...,45,blue,NaN,False,False,False,1,monster,water1,False
8,blastoise,blastoise,16,855,79,83,100,85,105,78,...,45,blue,NaN,False,False,False,1,monster,water1,False
9,caterpie,caterpie,3,29,45,30,35,20,20,45,...,255,green,NaN,False,False,False,1,bug,None,False


## Export / Import CSV

In [13]:
# Full df to CSV
df2 = df2.sort_values(by=['pokedex', 'name'])
df2.to_csv("pokemon_data.csv", index = False)

In [14]:
#Read Data From CSV
#df2 = pd.read_csv('.\pokemon_data.csv')

### Preview dataframes

In [15]:
pokedf.head()

,name,species,height,weight,hp,attack,defence,sp_attack,sp_defence,speed,type1,type2,ability1,ability2,hidden_ability
0,bulbasaur,bulbasaur,7,69,45,49,49,65,65,45,grass,poison,overgrow,chlorophyll,None
1,ivysaur,ivysaur,10,130,60,62,63,80,80,60,grass,poison,overgrow,chlorophyll,None
2,venusaur,venusaur,20,1000,80,82,83,100,100,80,grass,poison,overgrow,chlorophyll,None
3,charmander,charmander,6,85,39,52,43,60,50,65,fire,None,blaze,solar-power,None
4,charmeleon,charmeleon,11,190,58,64,58,80,65,80,fire,None,blaze,solar-power,None


In [16]:
specdf.head()

,name,pokedex,genus,capture_rate,color,shape,is_baby,is_legendary,is_mythical,generation,egg_group1,egg_group2
0,bulbasaur,1,NaN,45,green,NaN,False,False,False,1,monster,plant
1,ivysaur,2,NaN,45,green,NaN,False,False,False,1,monster,plant
2,venusaur,3,NaN,45,green,NaN,False,False,False,1,monster,plant
3,charmander,4,NaN,45,red,NaN,False,False,False,1,monster,dragon
4,charmeleon,5,NaN,45,red,NaN,False,False,False,1,monster,dragon


In [17]:
df2.head()

,name,species,height,weight,hp,attack,defence,sp_attack,sp_defence,speed,...,capture_rate,color,shape,is_baby,is_legendary,is_mythical,generation,egg_group1,egg_group2,is_mega
0,bulbasaur,bulbasaur,7,69,45,49,49,65,65,45,...,45,green,NaN,False,False,False,1,monster,plant,False
1,ivysaur,ivysaur,10,130,60,62,63,80,80,60,...,45,green,NaN,False,False,False,1,monster,plant,False
2,venusaur,venusaur,20,1000,80,82,83,100,100,80,...,45,green,NaN,False,False,False,1,monster,plant,False
1219,venusaur-gmax,venusaur,240,10000,80,82,83,100,100,80,...,45,green,NaN,False,False,False,1,monster,plant,False
1057,venusaur-mega,venusaur,24,1555,80,100,123,122,120,80,...,45,green,NaN,False,False,False,1,monster,plant,True


In [18]:
pokevalues = df2['species'].unique().tolist()
len(pokevalues)

1025

In [19]:
df2[df2['species'] != df2['name']]

,name,species,height,weight,hp,attack,defence,sp_attack,sp_defence,speed,...,capture_rate,color,shape,is_baby,is_legendary,is_mythical,generation,egg_group1,egg_group2,is_mega
1219,venusaur-gmax,venusaur,240,10000,80,82,83,100,100,80,...,45,green,NaN,False,False,False,1,monster,plant,False
1057,venusaur-mega,venusaur,24,1555,80,100,123,122,120,80,...,45,green,NaN,False,False,False,1,monster,plant,True
1220,charizard-gmax,charizard,280,10000,78,84,78,109,85,100,...,45,red,NaN,False,False,False,1,monster,dragon,False
1058,charizard-mega-x,charizard,17,1105,78,130,111,130,85,100,...,45,red,NaN,False,False,False,1,monster,dragon,True
1059,charizard-mega-y,charizard,17,1005,78,104,78,159,115,100,...,45,red,NaN,False,False,False,1,monster,dragon,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1299,ogerpon-cornerstone-mask,ogerpon,12,398,80,120,84,60,96,110,...,5,green,NaN,False,True,False,9,None,None,False
1298,ogerpon-hearthflame-mask,ogerpon,12,398,80,120,84,60,96,110,...,5,green,NaN,False,True,False,9,None,None,False
1297,ogerpon-wellspring-mask,ogerpon,12,398,80,120,84,60,96,110,...,5,green,NaN,False,True,False,9,None,None,False
1301,terapagos-stellar,terapagos,17,770,95,95,110,105,110,85,...,255,blue,NaN,False,True,False,9,None,None,False


In [20]:
df2.isnull().any()

name              False
species           False
height            False
weight            False
hp                False
attack            False
defence           False
sp_attack         False
sp_defence        False
speed             False
type1             False
type2              True
ability1          False
ability2           True
hidden_ability     True
pokedex           False
genus              True
capture_rate      False
color             False
shape              True
is_baby           False
is_legendary      False
is_mythical       False
generation        False
egg_group1         True
egg_group2         True
is_mega           False
dtype: bool

In [21]:
df2 = pd.read_csv('.\pokemon_data.csv')

# Testing Code (Disregard)

    t0 = time.time()
    #Pokemon df
    #url = f"https://pokeapi.co/api/v2/pokemon?limit=100000&offset=0"    #Real URL
    url = f"https://pokeapi.co/api/v2/pokemon?limit=100&offset=0"        #test URL
    response = requests.get(url)
    poke = response.json()
    for pokemon in poke['results']:
        get_pokemon_info(pokemon['url'])

    t1 = time.time()
    timer = t1-t0
    timer

    t0 = time.time()
    #url = f"https://pokeapi.co/api/v2/pokemon?limit=100000&offset=0"    #Real URL
    url = f"https://pokeapi.co/api/v2/pokemon?limit=100&offset=0"        #test URL
    response = requests.get(url)
    poke = response.json()
    for pokemon in poke['results']:
        get_pokemon_info2(pokemon['url'])

    t1 = time.time()
    timer = t1-t0
    timer

    name = []
    species = []
    height = []
    weight = []
    abilities = []
    types = []
    stats = []

    def get_pokemon_info2(url):
        response = requests.get(url)

        if response.status_code == 200:
            pokemon_data = response.json()

            name.append(pokemon_data["name"])
            species.append(pokemon_data["species"]["name"])
            height.append(pokemon_data["height"])
            weight.append(pokemon_data["weight"])
            abilities.append([ability["ability"]["name"] for ability in pokemon_data["abilities"]])
            types.append([stats_data["type"]["name"] for stats_data in pokemon_data["types"]])
            stats.append([stats_data["base_stat"] for stats_data in pokemon_data["stats"]])
                #"hp": pokemon_data["stats"][0]["base_stat"],
                #"attack": pokemon_data["stats"][1]["base_stat"],
                #"defence": pokemon_data["stats"][2]["base_stat"],
                #"sp_attack": pokemon_data["stats"][3]["base_stat"],
                #"sp_defence": pokemon_data["stats"][4]["base_stat"],
                #"speed": pokemon_data["stats"][5]["base_stat"],
            return None
        else:
            return None